A notebook that allows manual interactions with an RTSP server.  
Intended to increase one's understanding of the RTSP protocol.

In [10]:
import socket
import hashlib
from urllib.parse import urlparse

In [11]:
# Function to read RTSP URLs from a file
def read_rtsp_urls(filename):
    urls = []
    try:
        with open(filename, 'r') as file:
            urls = [line.strip() for line in file.readlines() if line.strip()]
    except FileNotFoundError:
        print(f"Error: File {filename} not found.")
    return urls

In [12]:
def remove_credentials_from_url(url):
    # Parse the RTSP URL to extract components
    parsed_url = urlparse(url)
    # Reconstruct the URL without username and password
    new_url = parsed_url._replace(netloc=parsed_url.hostname + (f":{parsed_url.port}" if parsed_url.port else ""))
    return new_url.geturl()

In [13]:
def send_rtsp_options(client_socket, url):
    
    try:
        # Prepare the RTSP OPTIONS request
        cseq = 1
        options_request = f"OPTIONS {url} RTSP/1.0\r\n"
        options_request += f"CSeq: {cseq}\r\n"
        options_request += "User-Agent: Python RTSP Client\r\n"
        options_request += "\r\n"
        
        # Send the request to the RTSP server
        client_socket.send(options_request.encode())
        
        # Receive the response from the server
        response = client_socket.recv(4096)
        print("RTSP Server Response:\n")
        print(response.decode())
        
    except Exception as e:
        print(f"Error: {e}")

In [14]:
def send_rtsp_describe(client_socket, url, username, password):
    global session_id, auth_realm, auth_nonce
    
    try:
        # Prepare the initial RTSP DESCRIBE request
        cseq = 2
        describe_request = f"DESCRIBE {url} RTSP/1.0\r\n"
        describe_request += f"CSeq: {cseq}\r\n"
        describe_request += "User-Agent: Python RTSP Client\r\n"
        describe_request += "Accept: application/sdp\r\n"
        describe_request += "\r\n"
        
        # Send the request to the RTSP server
        client_socket.send(describe_request.encode())
        
        # Receive the response from the server
        response = client_socket.recv(4096)
        response_str = response.decode()
        print("RTSP Server Response (DESCRIBE):\n")
        print(response_str)
        
        # Extract session ID if available
        session_id = None
        for line in response_str.split("\r\n"):
            if line.startswith("Session:"):
                session_id = line.split(" ")[1]
                break
        
        # Check if authentication is required (401 Unauthorized)
        if "401 Unauthorized" in response_str:
            # Extract the realm and nonce from the response
            auth_realm = None
            auth_nonce = None
            for line in response_str.split("\r\n"):
                if line.startswith("WWW-Authenticate: Digest"):
                    parts = line.split(",")
                    for part in parts:
                        if "realm" in part:
                            auth_realm = part.split('"')[1]
                        elif "nonce" in part:
                            auth_nonce = part.split('"')[1]
            
            if auth_realm and auth_nonce and username and password:
                # Compute the digest response
                ha1 = hashlib.md5(f"{username}:{auth_realm}:{password}".encode()).hexdigest()
                ha2 = hashlib.md5(f"DESCRIBE:{url}".encode()).hexdigest()
                response_digest = hashlib.md5(f"{ha1}:{auth_nonce}:{ha2}".encode()).hexdigest()
                
                # Prepare the authenticated DESCRIBE request
                cseq += 1
                describe_request = f"DESCRIBE {url} RTSP/1.0\r\n"
                describe_request += f"CSeq: {cseq}\r\n"
                describe_request += "User-Agent: Python RTSP Client\r\n"
                describe_request += "Accept: application/sdp\r\n"
                describe_request += (
                    f"Authorization: Digest username=\"{username}\", realm=\"{auth_realm}\", nonce=\"{auth_nonce}\", uri=\"{url}\", response=\"{response_digest}\"\r\n"
                )
                describe_request += "\r\n"
                
                # Send the authenticated request to the RTSP server
                client_socket.send(describe_request.encode())
                
                # Receive the response from the server
                response = client_socket.recv(4096)
                response_str = response.decode()
                print("RTSP Server Response (Authenticated DESCRIBE):\n")
                print(response_str)
                
                # Extract session ID if available
                for line in response_str.split("\r\n"):
                    if line.startswith("Session:"):
                        session_id = line.split(" ")[1]
                        break
            else:
                print("Authentication required, but realm/nonce extraction failed or credentials not provided.")
        
    except Exception as e:
        print(f"Error: {e}")


In [15]:
def send_rtsp_setup(client_socket, url, username, password):
    global session_id, auth_realm, auth_nonce
    
    try:
        # Prepare the RTSP SETUP request
        cseq = 4
        setup_request = f"SETUP {url}&trackID=0 RTSP/1.0\r\n"
        setup_request += f"CSeq: {cseq}\r\n"
        setup_request += "Transport: RTP/AVP;unicast;client_port=8000-8001\r\n"
        setup_request += "User-Agent: Python RTSP Client\r\n"
        if session_id:
            setup_request += f"Session: {session_id}\r\n"
        setup_request += "\r\n"
       
        # Check if authentication is required (401 Unauthorized)
        if True:
            # Use the previously obtained realm and nonce from DESCRIBE
            if auth_realm and auth_nonce and username and password:
                # Compute the digest response
                ha1 = hashlib.md5(f"{username}:{auth_realm}:{password}".encode()).hexdigest()
                ha2 = hashlib.md5(f"SETUP:{url}&trackID=1".encode()).hexdigest()
                response_digest = hashlib.md5(f"{ha1}:{auth_nonce}:{ha2}".encode()).hexdigest()
                
                # Prepare the authenticated SETUP request
                setup_request = f"SETUP {url}&trackID=1 RTSP/1.0\r\n"
                setup_request += f"CSeq: {cseq}\r\n"
                setup_request += "Transport: RTP/AVP;unicast;client_port=8000-8001\r\n"
                setup_request += "User-Agent: Python RTSP Client\r\n"
                if session_id:
                    setup_request += f"Session: {session_id}\r\n"
                setup_request += (
                    f"Authorization: Digest username=\"{username}\", realm=\"{auth_realm}\", nonce=\"{auth_nonce}\", uri=\"{url}&trackID=1\", response=\"{response_digest}\"\r\n"
                )
                setup_request += "\r\n"
                
                # Send the authenticated request to the RTSP server
                client_socket.send(setup_request.encode())
                
                # Receive the response from the server
                response = client_socket.recv(4096)
                print("RTSP Server Response (Authenticated SETUP):\n")
                print(response.decode())
            else:
                print("Setup failed")
                print("Authentication required, but realm/nonce extraction failed or credentials not provided.")
        
    except Exception as e:
        print(f"Error: {e}")

In [16]:
rtsp_urls = read_rtsp_urls('rtsp_urls.txt')

In [ ]:
# Parse the RTSP URL to extract IP address and port
parsed_url = urlparse(rtsp_urls[1][1:-1])
server_ip = parsed_url.hostname
server_port = parsed_url.port if parsed_url.port else 554
username = parsed_url.username
password = parsed_url.password
   
# Get a version of the url with no username or password.
url_with_no_cred = remove_credentials_from_url(rtsp_urls[1][1:-1])

# Create a TCP socket and make a connection to the server.
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((server_ip, server_port))

send_rtsp_options(client_socket, rtsp_urls[1][1:-1])
send_rtsp_describe(client_socket, rtsp_urls[1][1:-1], username, password)
send_rtsp_setup(client_socket, url_with_no_cred, username, password)

In [20]:
client_socket.close()